In [ ]:
import pandas as pd
import numpy as np
trn_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
tst_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
tst=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
fnl_trn=trn.merge(trn_trans,on='TransactionID',how='left')
fnl_tst=tst.merge(tst_trans,on='TransactionID',how='left')
dk={}
for i in range(1,39):
    if i<10:
        dk['id-0'+str(i)]='id_0'+str(i)
    else:
        dk['id-'+str(i)]='id_'+str(i)
fnl_tst=fnl_tst.rename(columns=dk)
import gc
del([trn_trans,trn,tst_trans,tst])
gc.collect()
fnl=pd.concat([fnl_trn,fnl_tst],0)
del([fnl_trn,fnl_tst])
gc.collect()
ids=list(fnl.filter(regex='id_'))
fnl[ids]=fnl[ids].fillna(-1)
fnl[['addr1','addr2']]=fnl[['addr1','addr2']].fillna(-1)
ids=list(fnl.filter(regex='card'))
fnl[ids]=fnl[ids].fillna(-1)
ids=list(fnl.filter(regex='V'))
fnl[ids]=fnl[ids].fillna(-1)
fnl[['addr1','addr2','DeviceInfo','DeviceType','P_emaildomain','R_emaildomain']]=fnl[['addr1','addr2','DeviceInfo','DeviceType','P_emaildomain','R_emaildomain']].fillna('nan')
ids=list(fnl.filter(regex='D'))
fnl[ids]=fnl[ids].fillna(-1)
ids=list(fnl.filter(regex='C'))
fnl[ids]=fnl[ids].fillna(0)
ids=list(fnl.filter(regex='M'))
fnl[ids]=fnl[ids].fillna(-1)
fnl[['dist1','dist2']]=fnl[['dist1','dist2']].fillna(-1)
import re
def number(data):
  try:
      return  ' '.join(re.findall(r'\d+',data)) 
  except:
    return -1
def charac(data):
    try:
      return ''.join(re.findall("[a-zA-Z]+", data))
    except:
        return -1
fnl['DeviceInfo_char']=list(map(charac,fnl['DeviceInfo'].fillna('nan')))
fnl['DeviceInfo_number']=list(map(number,fnl['DeviceInfo'].fillna('nan')))


fnl['DeviceType_char']=list(map(charac,fnl['DeviceType'].fillna('nan')))
fnl['DeviceType_number']=list(map(number,fnl['DeviceType'].fillna('nan')))


fnl['os_char']=list(map(charac,fnl['id_30'].fillna('nan')))
fnl['os_number']=list(map(number,fnl['id_30'].fillna('nan')))


fnl['browse_char']=list(map(charac,fnl['id_31'].fillna('nan')))
fnl['browse_number']=list(map(number,fnl['id_31'].fillna('nan')))


fnl['P_1']=fnl['P_emaildomain'].str.split('.').str[0]
fnl['P_2']=fnl['P_emaildomain'].str.split('.').str[1]


fnl['R_1']=fnl['R_emaildomain'].str.split('.').str[0]
fnl['R_2']=fnl['R_emaildomain'].str.split('.').str[1]
z=fnl.loc[~(fnl['isFraud'].isna())]
from tqdm import tqdm
from category_encoders.cat_boost import CatBoostEncoder
cols=list(fnl.select_dtypes(include=object))
for col in tqdm(cols):
    cbe=CatBoostEncoder(sigma=0.01)
    cbe.fit(z[col],z['isFraud'])
    fnl[col]=cbe.transform(fnl[col],fnl['isFraud'])
del(z)
gc.collect()
fnl.select_dtypes(include=object)
# used=['card'+str(i) for i in range(1,7)]
# used+=['C'+str(i) for i in range(1,15)]
# used+=['D'+str(i) for i in range(1,16)]
#fnl[used]=fnl[used].fillna(0)
fnl['day']=fnl['TransactionDT']//86400
fnl['month']=fnl['day']//30
fnl['week']=fnl['day']//7
# fnl[used]=fnl[used].fillna('nan')
# cats=['ProductCD','DeviceInfo_char','DeviceInfo_number','DeviceType_char','DeviceType_number','os_char','os_number','browse_char','browse_number','P_1','P_2','addr1']
# num=fnl
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
fnl=fnl.drop(['isFraud'],1)
fnl=reduce_mem_usage(fnl)
from sklearn.preprocessing import StandardScaler,MinMaxScaler
ss=StandardScaler()
fnl=pd.DataFrame(ss.fit_transform(fnl))
mms=MinMaxScaler()
fnl=pd.DataFrame(ss.fit_transform(fnl))
fnl=reduce_mem_usage(fnl)


In [ ]:
import gc
import os
gc.collect()
import numpy as np
import keras 
from math import ceil
from keras.models import Model
import matplotlib.pyplot as plt
from keras.utils import Sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Concatenate, Dropout,BatchNormalization
from keras.layers import Add
import keras.backend as K
class DAESequence(Sequence):
    def __init__(self,df,batch_size):
        self.batch_size=batch_size
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df.values
        self.idx=[]
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min((idx+1)*self.batch_size,self.len_data)
        idx=idx*self.batch_size
        size=last-idx
        output_x=self.data[idx:last]
        length=last-idx
        rnd=np.random.randint(0,self.len_data-size,size)
        rnd[rnd>idx]+=size
        cols=np.random.randint(0,self.columns,int(self.columns*0.1))
        noise_x=output_x.copy()
        noise_x[:,cols]=self.data[rnd[:,None],cols]
        return noise_x,output_x
from keras.models import Sequential
from keras.callbacks.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.layers import BatchNormalization
#fnl=pd.concat([fnl_trn,fnl_tst],0).select_dtypes(exclude='uint8')
batch_size=1024
checkpoint_path = "training_1.hdf5" 
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback =ModelCheckpoint(checkpoint_path, 
monitor='loss',save_best_only=True,save_weights_only=True,verbose=1)
len_input_columns=fnl.shape[1]
kernel_initializer_1=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
kernel_initializer_0=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.04499212706658476, seed=None)
kernel_initializer_2=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
model_dae = Sequential()
model_dae.add(Dense(units=len_input_columns,input_shape=(len_input_columns,), activation='relu', dtype='float32', name='Input'))
model_dae.add(Dense(units=int(300), activation='relu', dtype='float32',name='Hidden_1', kernel_initializer=kernel_initializer_0))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(200), activation='relu', dtype='float32',name='Hidden_2', kernel_initializer=kernel_initializer_1))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_3', kernel_initializer=kernel_initializer_2))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(25), activation='relu', dtype='float32',name='Hidden_4', kernel_initializer=kernel_initializer_2))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(2), activation='relu', dtype='float32',name='Hidden_5', kernel_initializer=kernel_initializer_2))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(25), activation='relu', dtype='float32',name='Hidden_6', kernel_initializer=kernel_initializer_2))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_7', kernel_initializer=kernel_initializer_2))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(200), activation='relu', dtype='float32',name='Hidden_8', kernel_initializer=kernel_initializer_2))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(300), activation='relu', dtype='float32',name='Hidden_9', kernel_initializer=kernel_initializer_2))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=len_input_columns, activation='linear', dtype='float32', name='Output'))
model_opt = keras.optimizers.SGD(lr=0.001, decay=1-0.9, momentum=0.7, nesterov=False)
model_dae.compile(loss='mean_squared_error',optimizer=model_opt)
hist=model_dae.fit_generator(DAESequence(fnl, batch_size=2048),steps_per_epoch=int(ceil(fnl.shape[0]/batch_size)),epochs=1000,workers=2)
model_dae.save_weights('autoenc.hdf5')